# Udacity: Self-Driving Car Engineer Nanodegree

## Project: Build a Traffic Sign Recognition Classifier

---
## Step 0: Load The Data

In [1]:
import pickle
import numpy as np
import tensorflow as tf
import csv
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

# Load pickled data
training_file = './traffic-signs-data/train.p'
validation_file= './traffic-signs-data/valid.p'
testing_file = './traffic-signs-data/test.p'
train_dict = pickle.load(open(training_file, mode='rb') )
valid_dict = pickle.load(open(validation_file, mode='rb'))
test_dict  = pickle.load(open(testing_file, mode='rb'))

X_train, y_train = train_dict['features'], train_dict['labels']
X_valid, y_valid = valid_dict['features'], valid_dict['labels']
X_test, y_test   = test_dict['features'], test_dict['labels']

Using TensorFlow backend.


----
## Step 1: Data augmentation functions

In [2]:
# Image processing library functions
from img_lib import list_images, rotate_image, translate_image, shear_image 
from img_lib import change_brightness_image, motion_blur_image

def random_transform_image(image):    
    if np.random.randint(2) == 0:
        return image
    
    transformation_library = ['rotation','translation','shear','brightness','blur']    
    transformation_id = transformation_library[np.random.randint(len(transformation_library))]
    
    if transformation_id == 'rotation':
        image = rotate_image(image)
        
    if transformation_id == 'translation':
        image = translate_image(image)
    
    if transformation_id == 'shear':
        image = shear_image(image)

    if transformation_id == 'brightness':
        image = change_brightness_image(image)
        
    if transformation_id == 'blur':
        image = motion_blur_image(image)
        
    if transformation_id == 'grayscale':
        image = gray_scale_image(image)
    
    return image

def generator_data(X_data):
    return np.array([random_transform_image(image) for image in X_data]).reshape(X_data.shape[0], 32,32,-1)


# Train the Model


In [3]:
from class_lenet import LeNet

n_classes = 43
model_fname = './OptimizedModel'
model = LeNet(n_classes)
model.compile(optimizer = tf.train.AdamOptimizer(learning_rate = 2e-3), activation_function = 'relu')
model.fit(x = X_train, y = y_train, batch_size = 64, 
          epochs = 2, 
          generator = generator_data,
          dropout_probabilities = {'keep_fc_3': 0.5, 'keep_fc_4': 0.5},
          validation_data = {'features': X_valid, 'labels': y_valid},
          save_trained_weights = model_fname, 
          verbose = False
         )
results = model.get_results_per_epoch()
training_loss_per_epoch = results['training_loss']
training_accuracy_per_epoch = results['training_accuracy']
validation_accuracy_per_epoch = results['validation_accuracy']
validation_loss_per_epoch = results['validation_loss']
print(np.max(validation_accuracy_per_epoch))



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

0.9086167800453515


# Evaluate model on test data


In [4]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,  model_fname)
    valid_results  = model.evaluate(model.preprocess_data(X_valid), y_valid)    
    print("validation errors = {}".format(len(valid_results["error_list"])))
    print("validation accuracy = {}".format(valid_results["accuracy"]))
    test_results    = model.evaluate(model.preprocess_data(X_test), y_test)
    print("test errors = {}".format(len(test_results["error_list"])))
    print("test accuracy = {}".format(test_results["accuracy"]))
    
    test_errors_rnd_idx = np.random.randint(len(test_results["error_list"]),size = 30)
    #list_images(valid_dict['features'][valid_results["error_list"]])
    #list_images(test_dict['features'][test_errors_rnd_idx])
    #plt.hist(test_dict['labels'][test_results["error_list"]])
    #plt.show()

INFO:tensorflow:Restoring parameters from ./OptimizedModel
validation errors = 204
validation accuracy = 0.9537414965986395
test errors = 726
test accuracy = 0.9425178147268408
